# Terms of Use

### Please solve the authorization problem of the dataset on your own. You shall be solely responsible for any problems caused by the use of non-authorized datasets for training and all consequences thereof.The repository and its maintainer, svc develop team, have nothing to do with the consequences!

1. This project is established for academic exchange purposes only and is intended for communication and learning purposes. It is not intended for production environments.
2. Any videos based on sovits that are published on video platforms must clearly indicate in the description that they are used for voice changing and specify the input source of the voice or audio, for example, using videos or audios published by others and separating the vocals as input source for conversion, which must provide clear original video or music links. If your own voice or other synthesized voices from other commercial vocal synthesis software are used as the input source for conversion, you must also explain it in the description.
3. You shall be solely responsible for any infringement problems caused by the input source. When using other commercial vocal synthesis software as input source, please ensure that you comply with the terms of use of the software. Note that many vocal synthesis engines clearly state in their terms of use that they cannot be used for input source conversion.
4. Continuing to use this project is deemed as agreeing to the relevant provisions stated in this repository README. This repository README has the obligation to persuade, and is not responsible for any subsequent problems that may arise.
5. If you distribute this repository's code or publish any results produced by this project publicly (including but not limited to video sharing platforms), please indicate the original author and code source (this repository).
6. If you use this project for any other plan, please contact and inform the author of this repository in advance. Thank you very much.


## **Note:**
## **Make sure there is no a directory named `sovits4data` in your google drive at the first time you use this notebook.**
## **It will be created to store some necessary files.**
## **For sure you can change it to another directory by modifying `sovits_data_dir` variable.**

# **Initialize environment**

In [1]:
#@title Connect to colab runtime and check GPU

#@markdown # Connect to colab runtime and check GPU

#@markdown

!nvidia-smi

Sun Mar 30 20:03:20 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   31C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [26]:
!apt update
!apt install python3.10

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
41 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InR

In [29]:
!python3 --version
!python3.10 --version

Python 3.10.12
Python 3.10.12


In [28]:
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 2
!sudo update-alternatives --config python3

There are 2 choices for the alternative python3 (providing /usr/bin/python3).

  Selection    Path                 Priority   Status
------------------------------------------------------------
* 0            /usr/bin/python3.11   2         auto mode
  1            /usr/bin/python3.10   2         manual mode
  2            /usr/bin/python3.11   2         manual mode

Press <enter> to keep the current choice[*], or type selection number: 1
update-alternatives: using /usr/bin/python3.10 to provide /usr/bin/python3 (python3) in manual mode


In [30]:
# prompt: how to install pip, i dont have pip command working

!apt-get update -y
!apt-get install python3-pip -y


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [31]:
#@title Clone repository and install requirements

#@markdown # Clone repository and install requirements

#@markdown

#@markdown ### After the execution is completed, the runtime will **automatically restart**

#@markdown

!git clone https://github.com/svc-develop-team/so-vits-svc -b 4.1-Stable
# %cd /content/so-vits-svc
# os.chdir('so-vits-svc')
%pip install --upgrade pip setuptools
!cat requirements.txt | xargs -n 1 pip install --extra-index-url https://download.pytorch.org/whl/cu118
!pip install praat-parselmouth
!pip install ipywidgets
!pip install huggingface_hub
!pip install pip==23.0.1 # fix pip version for fairseq install
!pip install fairseq==0.12.2
# exit()

fatal: destination path 'so-vits-svc' already exists and is not an empty directory.
  Using cached pip-25.0.1-py3-none-any.whl (1.8 MB)
  Using cached setuptools-78.1.0-py3-none-any.whl (1.3 MB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 59.6.0
    Not uninstalling setuptools at /usr/lib/python3/dist-packages, outside environment /usr
    Can't uninstall 'setuptools'. No files were found to uninstall.
  Attempting uninstall: pip
    Found existing installation: pip 22.0.2
    Not uninstalling pip at /usr/lib/python3/dist-packages, outside environment /usr
    Can't uninstall 'pip'. No files were found to uninstall.


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
  Using cached ffmpeg_python-0.2.0-py3-none-any.whl.metadata (1.7 kB)
Using cached ffmpeg_python-0.2.0-py3-none-any.whl (25 kB)
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 2.0.1
    Uninstalling MarkupSafe-2.0.1:
      Successfully uninstalled MarkupSafe-2.0.1
  Attempting uninstall: blinker
    Found existing installation: blinker 1.4
error: uninstall-distutils-installed-package

× Cannot uninstall blinker 1.4
╰─> It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
  Using cached flask_cors-5.0.1-py3-none-any.whl.metadata (961 bytes)
  Using cached flask-3.1.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached 

  Using cached pip-23.0.1-py3-none-any.whl.metadata (4.1 kB)
Using cached pip-23.0.1-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 25.0.1
    Uninstalling pip-25.0.1:
      Successfully uninstalled pip-25.0.1
  Using cached fairseq-0.12.2.tar.gz (9.6 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Cython-3.0.12-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Using cached sacrebleu-2.5.1-py3-none-any.whl (104 kB)
  Using cached omegaconf-2.0.6-py3-none-any.whl (36 kB)
  Using cached hydra_core-1.0.7-py3-none-any.whl (123 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.8/298.8 kB 3.9 MB/s eta 0:00:00
  Using cached antlr4-python3-runtime-4.8.tar.gz (112 kB)
  Preparing metadata (setup.py) ... done
  Using cached portalocker-3.1.1-py3-none-any.whl (19 kB)
     ━━━━━━━━━━

In [32]:
#@title Mount google drive and select which directories to sync with google drive

#@markdown # Mount google drive and select which directories to sync with google drive

#@markdown

from google.colab import drive
drive.mount("/content/drive")

#@markdown Directory to store **necessary files**, dont miss the slash at the end👇.
sovits_data_dir = "/content/drive/MyDrive/sovits4data/"  #@param {type:"string"}
#@markdown By default it will create a `sovits4data/` folder in your google drive.
RAW_DIR = sovits_data_dir + "raw/"
RESULTS_DIR = sovits_data_dir + "results/"
FILELISTS_DIR = sovits_data_dir + "filelists/"
CONFIGS_DIR = sovits_data_dir + "configs/"
LOGS_DIR = sovits_data_dir + "logs/44k/"

#@markdown

#@markdown ### These folders will be synced with your google drvie

#@markdown　### **Strongly recommend to check all.**

#@markdown Sync **input audios** and **output audios**
sync_raw_and_results = True  #@param {type:"boolean"}
if sync_raw_and_results:
  !mkdir -p {RAW_DIR}
  !mkdir -p {RESULTS_DIR}
  !rm -rf /content/so-vits-svc/raw
  !rm -rf /content/so-vits-svc/results
  !ln -s {RAW_DIR} /content/so-vits-svc/raw
  !ln -s {RESULTS_DIR} /content/so-vits-svc/results

#@markdown Sync **config** and **models**
sync_configs_and_logs = True  #@param {type:"boolean"}
if sync_configs_and_logs:
  !mkdir -p {FILELISTS_DIR}
  !mkdir -p {CONFIGS_DIR}
  !mkdir -p {LOGS_DIR}
  !rm -rf /content/so-vits-svc/filelists
  !rm -rf /content/so-vits-svc/configs
  !rm -rf /content/so-vits-svc/logs/44k
  !ln -s {FILELISTS_DIR} /content/so-vits-svc/filelists
  !ln -s {CONFIGS_DIR} /content/so-vits-svc/configs
  !ln -s {LOGS_DIR} /content/so-vits-svc/logs/44k

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
#@title Get pretrained model(Optional but strongly recommend).

#@markdown # Get pretrained model(Optional but strongly recommend).

#@markdown

#@markdown - Pre-trained model files: `G_0.pth` `D_0.pth`
#@markdown   - Place them under /sovits4data/logs/44k/ in your google drive manualy

#@markdown Get them from svc-develop-team(TBD) or anywhere else.

#@markdown Although the pretrained model generally does not cause any copyright problems, please pay attention to it. For example, ask the author in advance, or the author has indicated the feasible use in the description clearly.

download_pretrained_model = True #@param {type:"boolean"}
D_0_URL = "https://huggingface.co/datasets/ms903/sovits4.0-768vec-layer12/resolve/main/sovits_768l12_pre_large_320k/clean_D_320000.pth" #@param ["https://huggingface.co/datasets/ms903/sovits4.0-768vec-layer12/resolve/main/sovits_768l12_pre_large_320k/clean_D_320000.pth", "https://huggingface.co/1asbgdh/sovits4.0-volemb-vec768/resolve/main/clean_D_320000.pth", "https://huggingface.co/datasets/ms903/sovits4.0-768vec-layer12/resolve/main/vol_emb/clean_D_320000.pth"] {allow-input: true}
G_0_URL = "https://huggingface.co/datasets/ms903/sovits4.0-768vec-layer12/resolve/main/sovits_768l12_pre_large_320k/clean_G_320000.pth" #@param ["https://huggingface.co/datasets/ms903/sovits4.0-768vec-layer12/resolve/main/sovits_768l12_pre_large_320k/clean_G_320000.pth", "https://huggingface.co/1asbgdh/sovits4.0-volemb-vec768/resolve/main/clean_G_320000.pth", "https://huggingface.co/datasets/ms903/sovits4.0-768vec-layer12/resolve/main/vol_emb/clean_G_320000.pth"] {allow-input: true}

download_pretrained_diffusion_model = True #@param {type:"boolean"}
diff_model_URL = "https://huggingface.co/datasets/ms903/Diff-SVC-refactor-pre-trained-model/resolve/main/fix_pitch_add_vctk_600k/model_0.pt" #@param {type:"string"}

%cd /content/so-vits-svc

if download_pretrained_model:
    !curl -L {D_0_URL} -o logs/44k/D_0.pth
    !md5sum logs/44k/D_0.pth
    !curl -L {G_0_URL} -o logs/44k/G_0.pth
    !md5sum logs/44k/G_0.pth

if download_pretrained_diffusion_model:
    !mkdir -p logs/44k/diffusion
    !curl -L {diff_model_URL} -o logs/44k/diffusion/model_0.pt
    !md5sum logs/44k/diffusion/model_0.pt

/content/so-vits-svc
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1115  100  1115    0     0   6328      0 --:--:-- --:--:-- --:--:--  6335
100  178M  100  178M    0     0  65.3M      0  0:00:02  0:00:02 --:--:-- 74.0M
791dfe20d626a7eca67c5c10420d327f  logs/44k/D_0.pth
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1111  100  1111    0     0   7314      0 --:--:-- --:--:-- --:--:--  7357
100  199M  100  199M    0     0  67.8M      0  0:00:02  0:00:02 --:--:-- 78.8M
8249cd8bbfd6a69ffad5125ad169c70f  logs/44k/G_0.pth
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1091  100  1091    0     0   6384      0 --:--:-- --:--:-- --:--:--  6417
100  210

# **Dataset preprocessing**

Pack and upload your raw dataset(dataset_raw/) to your google drive.

Makesure the file structure in your zip file looks like this:

```
YourZIPforSingleSpeakers.zip
└───speaker
    ├───xxx1-xxx1.wav
    ├───...
    └───Lxx-0xx8.wav
```

```
YourZIPforMultipleSpeakers.zip
├───speaker0
│   ├───xxx1-xxx1.wav
│   ├───...
│   └───Lxx-0xx8.wav
└───speaker1
    ├───xx2-0xxx2.wav
    ├───...
    └───xxx7-xxx007.wav
```

**Even if there is only one speaker, a folder named `{speaker_name}` is needed.**

In [18]:
#@title Get raw dataset from google drive

#@markdown # Get raw dataset from google drive

#@markdown

#@markdown Directory where **your zip file** located in, dont miss the slash at the end👇.
sovits_data_dir = "/content/drive/MyDrive/sovits4data/"  #@param {type:"string"}
#@markdown Filename of **your zip file**, do NOT be "dataset.zip"
zip_filename = "YourZIPFilenameofRawDataset.zip"  #@param {type:"string"}
ZIP_PATH = sovits_data_dir + zip_filename

!unzip -od /content/so-vits-svc/dataset_raw {ZIP_PATH}

unzip:  cannot find or open /content/drive/MyDrive/sovits4data/YourZIPFilenameofRawDataset.zip, /content/drive/MyDrive/sovits4data/YourZIPFilenameofRawDataset.zip.zip or /content/drive/MyDrive/sovits4data/YourZIPFilenameofRawDataset.zip.ZIP.


In [21]:
!cp -r /content/drive/MyDrive/dataset/saotome  /content/so-vits-svc/dataset_raw

In [22]:
#@title Resample to 44100Hz and mono

#@markdown # Resample to 44100Hz and mono

#@markdown

%cd /content/so-vits-svc
!python resample.py

/content/so-vits-svc
CPU count: 2
./dataset_raw/saotome
resampling: ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:06:10


In [23]:
#@title Divide filelists and generate config.json

#@markdown # Divide filelists and generate config.json

#@markdown

%cd /content/so-vits-svc

speech_encoder = "vec768l12" #@param ["vec768l12", "vec256l9", "hubertsoft", "whisper-ppg", "whisper-ppg-large"]
use_vol_aug = False #@param {type:"boolean"}
vol_aug = "--vol_aug" if use_vol_aug else ""

from pretrain.meta import download_dict
download_dict = download_dict()

url = download_dict[speech_encoder]["url"]
output = download_dict[speech_encoder]["output"]

import os
if not os.path.exists(output):
  !curl -L {url} -o {output}
  !md5sum {output}

!python preprocess_flist_config.py --speech_encoder={speech_encoder} {vol_aug}

/content/so-vits-svc
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    10    0    10    0     0     18      0 --:--:-- --:--:-- --:--:--    18
100 1268M  100 1268M    0     0  24.1M      0  0:00:52  0:00:52 --:--:-- 28.0M
c2287f26ea98387317897a4ca4de66d4  ./pretrain/checkpoint_best_legacy_500.pt
  0% 0/1 [00:00<?, ?it/s]2025-03-30 20:34:33.958 | WARNING  | __main__:<module>:59 - Detected non-ASCII file name: ./dataset/44k/saotome/cleaned_j0679 .wav
2025-03-30 20:34:33.959 | WARNING  | __main__:<module>:59 - Detected non-ASCII file name: ./dataset/44k/saotome/cleaned_j1639 .wav
100% 1/1 [00:00<00:00, 16.73it/s]
2025-03-30 20:34:33.998 | INFO     | __main__:<module>:74 - Writing ./filelists/train.txt
100% 2422/2422 [00:00<00:00, 1211521.08it/s]
2025-03-30 20:34:34.349 | INFO     | __main__:<module

In [35]:
!pip uninstall torch -y

Found existing installation: torch 2.6.0+cu118
Uninstalling torch-2.6.0+cu118:
  Successfully uninstalled torch-2.6.0+cu118


In [39]:
!wget https://developer.download.nvidia.com/compute/cuda/11.8.0/local_installers/cuda_11.8.0_520.61.05_linux.run
!sh cuda_11.8.0_520.61.05_linux.run --silent --toolkit

--2025-03-30 21:26:18--  https://developer.download.nvidia.com/compute/cuda/11.8.0/local_installers/cuda_11.8.0_520.61.05_linux.run
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 23.59.88.207, 23.59.88.195
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|23.59.88.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4336730777 (4.0G) [application/octet-stream]
Saving to: ‘cuda_11.8.0_520.61.05_linux.run’

cuda_11.8.0_520.61. 100%[===================>]   4.04G   201MB/s    in 19s     

2025-03-30 21:26:37 (217 MB/s) - ‘cuda_11.8.0_520.61.05_linux.run’ saved [4336730777/4336730777]



In [40]:
!export PATH=/usr/local/cuda-11.8/bin:$PATH
!export LD_LIBRARY_PATH=/usr/local/cuda-11.8/lib64:$LD_LIBRARY_PATH

In [36]:
!pip install torch==2.5.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [42]:
!apt update
!install cuda-11-8 -y

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
41 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InR

In [41]:
#@title Generate hubert and f0

#@markdown # Generate hubert and f0

#@markdown
%cd /content/so-vits-svc

f0_predictor = "rmvpe" #@param ["crepe", "pm", "dio", "harvest", "rmvpe", "fcpe"]
use_diff = True #@param {type:"boolean"}

import os
if f0_predictor == "rmvpe" and not os.path.exists("./pretrain/rmvpe.pt"):
  !curl -L https://huggingface.co/datasets/ylzz1997/rmvpe_pretrain_model/resolve/main/rmvpe.pt -o pretrain/rmvpe.pt

if f0_predictor == "fcpe" and not os.path.exists("./pretrain/fcpe.pt"):
  !curl -L https://huggingface.co/datasets/ylzz1997/rmvpe_pretrain_model/resolve/main/fcpe.pt -o pretrain/fcpe.pt


diff_param = ""
if use_diff:
  diff_param = "--use_diff"

  if not os.path.exists("./pretrain/nsf_hifigan/model"):
    !curl -L https://github.com/openvpi/vocoders/releases/download/nsf-hifigan-v1/nsf_hifigan_20221211.zip -o nsf_hifigan_20221211.zip
    !md5sum nsf_hifigan_20221211.zip
    !unzip nsf_hifigan_20221211.zip
    !rm -rf pretrain/nsf_hifigan
    !mv -v nsf_hifigan pretrain

!python preprocess_hubert_f0.py --f0_predictor={f0_predictor} {diff_param}

/content/so-vits-svc
Traceback (most recent call last):
  File "/content/so-vits-svc/preprocess_hubert_f0.py", line 18, in <module>
    from diffusion.vocoder import Vocoder
  File "/content/so-vits-svc/diffusion/vocoder.py", line 2, in <module>
    from torchaudio.transforms import Resample
  File "/usr/local/lib/python3.10/dist-packages/torchaudio/__init__.py", line 2, in <module>
    from . import _extension  # noqa  # usort: skip
  File "/usr/local/lib/python3.10/dist-packages/torchaudio/_extension/__init__.py", line 38, in <module>
    _load_lib("libtorchaudio")
  File "/usr/local/lib/python3.10/dist-packages/torchaudio/_extension/utils.py", line 60, in _load_lib
    torch.ops.load_library(path)
  File "/usr/local/lib/python3.10/dist-packages/torch/_ops.py", line 1350, in load_library
    ctypes.CDLL(path)
  File "/usr/lib/python3.10/ctypes/__init__.py", line 374, in __init__
    self._handle = _dlopen(self._name, mode)
OSError: libcudart.so.11.0: cannot open shared object file: N

In [ ]:
#@title Save the preprocessed dataset to google drive

#@markdown # Save the preprocessed dataset to google drive

#@markdown

#@markdown You can save the dataset and related files to your google drive for the next training

#@markdown **Directory for saving**, dont miss the slash at the end👇.
sovits_data_dir = "/content/drive/MyDrive/sovits4data/" #@param {type:"string"}

#@markdown There will be a `dataset.zip` contained `dataset/` in your google drive, which is preprocessed data.

!mkdir -p {sovits_data_dir}
!zip -r dataset.zip /content/so-vits-svc/dataset
!cp -vr dataset.zip "{sovits_data_dir}"

In [ ]:
#@title Unzip preprocessed dataset from google drive directly if you have preprocessed already.

#@markdown # Unzip preprocessed dataset from google drive directly if you have preprocessed already.

#@markdown

#@markdown Directory where **your preprocessed dataset** located in, dont miss the slash at the end👇.
sovits_data_dir = "/content/drive/MyDrive/sovits4data/" #@param {type:"string"}
CONFIG = sovits_data_dir + "configs/"
FILELISTS = sovits_data_dir + "filelists/"
DATASET = sovits_data_dir + "dataset.zip"

!cp -vr {CONFIG} /content/so-vits-svc/
!cp -vr {FILELISTS} /content/so-vits-svc/
!unzip {DATASET} -d /

# **Trainning**

In [ ]:
#@title Start training

#@markdown # Start training

#@markdown If you want to use pre-trained models, upload them to /sovits4data/logs/44k/ in your google drive manualy.

#@markdown

%cd /content/so-vits-svc

#@markdown Whether to enable tensorboard
tensorboard_on = True  #@param {type:"boolean"}

if tensorboard_on:
  %load_ext tensorboard
  %tensorboard --logdir logs/44k

config_path = "configs/config.json"

from pretrain.meta import get_speech_encoder
url, output = get_speech_encoder(config_path)

import os
if not os.path.exists(output):
  !curl -L {url} -o {output}

!python train.py -c {config_path} -m 44k

In [ ]:
#@title Train cluster model (Optional)

#@markdown # Train cluster model (Optional)

#@markdown #### Details see [README.md#cluster-based-timbre-leakage-control](https://github.com/svc-develop-team/so-vits-svc#cluster-based-timbre-leakage-control)

#@markdown

%cd /content/so-vits-svc
!python cluster/train_cluster.py --gpu

In [ ]:
#@title Train index model (Optional)

#@markdown # Train index model (Optional)

#@markdown #### Details see [README.md#feature-retrieval](https://github.com/svc-develop-team/so-vits-svc#feature-retrieval)

#@markdown

%cd /content/so-vits-svc
!python train_index.py -c configs/config.json

In [ ]:
#@title Train diffusion model (Optional)

#@markdown # Train diffusion model (Optional)

#@markdown #### Details see [README.md#-about-shallow-diffusion](https://github.com/svc-develop-team/so-vits-svc#-about-shallow-diffusion)

#@markdown

%cd /content/so-vits-svc

import os
if not os.path.exists("./pretrain/nsf_hifigan/model"):
  !curl -L https://github.com/openvpi/vocoders/releases/download/nsf-hifigan-v1/nsf_hifigan_20221211.zip -o nsf_hifigan_20221211.zip
  !unzip nsf_hifigan_20221211.zip
  !rm -rf pretrain/nsf_hifigan
  !mv -v nsf_hifigan pretrain

#@markdown Whether to enable tensorboard
tensorboard_on = True  #@param {type:"boolean"}

if tensorboard_on:
  %load_ext tensorboard
  %tensorboard --logdir logs/44k

!python train_diff.py -c configs/diffusion.yaml

# keep colab alive
Open the devtools and copy & paste to run the scrips.


```JavaScript
const ping = () => {
  const btn = document.querySelector("colab-connect-button");
  const inner_btn = btn.shadowRoot.querySelector("#connect");
  if (inner_btn) {
    inner_btn.click();
    console.log("Clicked on connect button");
  } else {
    console.log("connect button not found");
  }

  const nextTime = 50000 + Math.random() * 10000;

  setTimeout(ping, nextTime);
};

ping();
```

# **Inference**
### Upload wav files from this notebook
### **OR**
### Upload to `sovits4data/raw/` in your google drive manualy (should be faster)

In [ ]:
#title Download nsf_hifigan if you need it

%cd /content/so-vits-svc
!curl -L https://github.com/openvpi/vocoders/releases/download/nsf-hifigan-v1/nsf_hifigan_20221211.zip -o /content/so-vits-svc/nsf_hifigan_20221211.zip
!unzip nsf_hifigan_20221211.zip
!rm -rf pretrain/nsf_hifigan
!mv -v nsf_hifigan pretrain


In [ ]:
#@title Upload wav files, the filename should not contain any special symbols like `#` `$` `(` `)`

#@markdown # Upload wav files, the filename should not contain any special symbols like `#` `$` `(` `)`

#@markdown

%cd /content/so-vits-svc
%run wav_upload.py --type audio

In [ ]:
#@title Start inference (and download)

#@markdown # Start inference (and download)

#@markdown Parameters see [README.MD#Inference](https://github.com/svc-develop-team/so-vits-svc#-inference)

#@markdown

wav_filename = "YourWAVFile.wav"  #@param {type:"string"}
model_filename = "G_210000.pth"  #@param {type:"string"}
model_path = "/content/so-vits-svc/logs/44k/" + model_filename
speaker = "YourSpeaker"  #@param {type:"string"}
trans = "0"  #@param {type:"string"}
cluster_infer_ratio = "0"  #@param {type:"string"}
auto_predict_f0 = False  #@param {type:"boolean"}
apf = ""
if auto_predict_f0:
  apf = " -a "

f0_predictor = "crepe" #@param ["crepe", "pm", "dio", "harvest", "rmvpe", "fcpe"]

enhance = False  #@param {type:"boolean"}
ehc = ""
if enhance:
  ehc = " -eh "
#@markdown

#@markdown Generally keep default:
config_filename = "config.json"  #@param {type:"string"}
config_path = "/content/so-vits-svc/configs/" + config_filename

from pretrain.meta import get_speech_encoder
url, output = get_speech_encoder(config_path)

import os

if f0_predictor == "rmvpe" and not os.path.exists("./pretrain/rmvpe.pt"):
  !curl -L https://huggingface.co/datasets/ylzz1997/rmvpe_pretrain_model/resolve/main/rmvpe.pt -o pretrain/rmvpe.pt

if f0_predictor == "fcpe" and not os.path.exists("./pretrain/fcpe.pt"):
  !curl -L https://huggingface.co/datasets/ylzz1997/rmvpe_pretrain_model/resolve/main/fcpe.pt -o pretrain/fcpe.pt

if not os.path.exists(output):
  !curl -L {url} -o {output}

kmeans_filenname = "kmeans_10000.pt"  #@param {type:"string"}
kmeans_path = "/content/so-vits-svc/logs/44k/" + kmeans_filenname
slice_db = "-40"  #@param {type:"string"}
wav_format = "flac"  #@param {type:"string"}

key = "auto" if auto_predict_f0 else f"{trans}key"
cluster_name = "" if cluster_infer_ratio == "0" else f"_{cluster_infer_ratio}"
isdiffusion = "sovits"
wav_output = f"/content/so-vits-svc/results/{wav_filename}_{key}_{speaker}{cluster_name}_{isdiffusion}_{f0_predictor}.{wav_format}"

%cd /content/so-vits-svc
!python inference_main.py -n {wav_filename} -m {model_path} -s {speaker} -t {trans} -cr {cluster_infer_ratio} -c {config_path} -cm {kmeans_path} -sd {slice_db} -wf {wav_format} {apf} --f0_predictor={f0_predictor} {ehc}

#@markdown

#@markdown If you dont want to download from here, uncheck this.
download_after_inference = True  #@param {type:"boolean"}

if download_after_inference:
  from google.colab import files
  files.download(wav_output)